# Redis Playground

Iremos por encima de varias llamadas al API intentando hacer una copia de una red social.

La libreria oficial de redis para python provee un API casi 1:1 con el cli de redis.

## Setup First

In [ ]:
from redis import StrictRedis
redis = StrictRedis(host='redis')

## Primero un par de conceptos basicos

In [ ]:
redis.set('clave', 'valor')
redis.get('clave')

In [ ]:
redis.set('clave', 2)

In [ ]:
redis.get('clave')

### Operaciones atomicas

Redis garantiza atomicidad completa en sus operaciones.

In [ ]:
redis.set('visitas', 100)
visitas = int(redis.get('visitas'))
visitas += 1
redis.set('visitas', visitas)
redis.get('visitas')

In [ ]:
#redis.set('visitas', 100)
redis.incrby('visitas', int(redis.get('tasa_de_vuelta')*100)
redis.get('visitas')

### Listas

In [ ]:
redis.lpush('lista', 'a')
redis.lpush('lista', 'b')
redis.lpush('lista', 'c')

In [ ]:
redis.lrange('lista', 0, 1) # LRANGE lista 0 1

In [ ]:
redis.lrange('lista', 0, -1) # LRANGE lista 0 -1

In [ ]:
redis.rpush('lista', 'd')
redis.lrange('lista', 0, -1)

### Sets

In [ ]:
redis.sadd('set', 'a')
redis.sadd('set', 'b')
redis.sadd('set', 1)
redis.sadd('set', 2)
redis.sadd('set', "True")

In [ ]:
redis.scard('set')

In [ ]:
redis.smembers('set')

In [ ]:
redis.sadd('set', 1)

In [ ]:
redis.smembers('set')

In [ ]:
redis.sadd('set_nuevo', 1)
redis.sadd('set_nuevo', 2)
redis.sadd('set_nuevo', "False")

In [ ]:
redis.sinter('set', 'set_nuevo')

In [ ]:
redis.sismember('set', 56)

### Sorted Sets

In [ ]:
redis.zadd('set_ordenado', {'a': 0.0}) # ZADD set_ordenado 0 a
redis.zadd('set_ordenado', {'b': 5.0})
redis.zadd('set_ordenado', {'c': 10.0})

In [ ]:
redis.zrange('set_ordenado', 0, -1)

In [ ]:
redis.zrevrange('set_ordenado', 0, -1, withscores=True)

In [ ]:
redis.zscore('set_ordenado', 'b')

In [ ]:
redis.zscore('set_ordenado', 'd')

### Hashes

In [ ]:
redis.hmset('user_hash', {"name": "Ricardo", "surname": "Vegas", "age": 32})

In [ ]:
redis.hmget('user_hash', ["age", "surname"])

In [ ]:
redis.hget('user_hash', 'name')

In [ ]:
redis.hdel('user_hash', 'age')

In [ ]:
redis.hmget('user_hash', ["age", "surname"])

In [ ]:
redis.hgetall('user_hash')

In [ ]:
redis.delete('user_hash')

In [ ]:
redis.hgetall('user_hash')

## Una red social en REDIS??

Con lo que sabemos, es posible montar gran parte de una red social, veamos:

In [ ]:
# Una secuencia para los user_id
redis.incr('next_user_id', 1000)

In [ ]:
# Obtenemos un user id y registramos al usuario en una key propia (1 por usuario)
user_id = redis.incr('next_user_id')
redis.hmset('user' + str(user_id), {"username": "rvegas", "password": "123password"})

In [ ]:
redis.hgetall('user' + str(user_id))

In [ ]:
# Guardamos la relacion de cada usuario con su id, para construir la key propia de sus datos
redis.hset('user_ids', "rvegas", int(user_id))

In [ ]:
# en user_ids tendriamos los ids de todos los usuarios, no sus datos
redis.hgetall('user_ids')

In [ ]:
# Si en otra ejecucion quiero los datos de un usuario (LOGIN)
current_user_id = int(redis.hget('user_ids', 'rvegas'))
redis.hgetall('user' + str(current_user_id))

In [ ]:
# Como guardo los followers? con sets ordenados!
import time
redis.zadd('followers' + str(current_user_id), {'mariav': time.time()})

In [ ]:
redis.zadd('followers' + str(current_user_id), {'luisjc': time.time()})

In [ ]:
followers_key = 'followers' + str(current_user_id)
redis.zrange(followers_key, 0, -1, withscores=True)

In [ ]:
# Como guardo los posts?? con listas!

posts_sequence = redis.incr('post_ids', 1000)
next_post_id = redis.incr('post_ids')
new_post = {"content": "I love Redis!"}

redis.hmset('posts' + str(next_post_id), new_post)

user_posts_key = 'posts' + str(current_user_id)
redis.lpush(user_posts_key, str(next_post_id))


In [ ]:
user_posts = redis.lrange(user_posts_key, 0, -1)

In [ ]:
redis.hgetall('posts' + str(int(user_posts[0])))

## Finalmente, MUY IMPORTANTE! Expirations!!!

Supongamos que queremos dar api keys a los usuarios cuando hacen login, pero queremos que las keys 
tengan validez de horas:

In [ ]:
import uuid
#user_keys:1001 (mi user es 1001, mi token esta en user_keys:1001, mis datos estan en user_data:1001)
redis.setex('user_keys:' + str(current_user_id), 10, str(uuid.uuid4()))

In [ ]:
redis.get('user_keys:' + str(current_user_id))